In [ ]:
import csaf.config as cconf
import csaf.system as csys

# create a csaf configuration out of toml
my_conf = cconf.SystemConfig.from_toml("/csaf-system/f16_shield_config.toml")

In [ ]:
# create pub/sub components out of the configuration
my_system = csys.System.from_config(my_conf)

simulation_timespan = [0, 35.0]

# simulate and collect time traces out of the components
trajs = my_system.simulate_tspan(simulation_timespan, show_status=True)

# destroy components and unbind all used sockets
my_system.unbind() 

In [ ]:
import sys
sys.path.append("/csaf-system/components")

import acas.kernel as akern
import acas.trajectory as atraj
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
x, y = np.array(trajs['plant'].states)[:, 9], np.array(trajs['plant'].states)[:, 10]
z = np.array(trajs['plant'].states)[:, 11]
t = trajs['plant'].times

In [ ]:
nsl = 800
st = 20

# intent points
waypoints = [[20, 1100, 20.0], [35, 1400, 0.0]]
# uncertainty around intent
sigmas = [[400, 10], [300, 1]]

# training data
tt, xt, yt, zt = t[:nsl:st], x[:nsl:st], y[:nsl:st], z[:nsl:st]
zd = np.diff(zt)/np.diff(tt)
zd = np.hstack((zd[0], zd))

# append intent to training data
for ti, zi, zdi in waypoints:
    tt = np.hstack((tt, ti))
    zt = np.hstack((zt, zi))
    zd = np.hstack((zd, zdi))

In [ ]:

te = atraj.CubicSplineTrajectoryEstimator(11.0, 50.0, 50.0)

In [ ]:
sigma = np.ones(len(tt))*0.01
sigmap = np.ones(len(tt))*0.01

for idx, (sigi, sigpi) in enumerate(sigmas[::-1]):
    sigma[-idx-1] = sigi
    sigmap[-idx-1] = sigpi

te.fit(tt, np.vstack((zt, zd)).T, sigma=sigma, sigmap=sigmap)

In [ ]:
N = 4000
mu = [te.mu(ti) for ti in t[:N]]
va = [te.covariance(ti, ti) for ti in t[:N]]

In [ ]:
def saturate(ina):
    return [(0.0 if inai <= 0.0 else inai) for inai in ina]

plt.figure(figsize=(8, 6))
plt.plot(t[:N], z[:N], label='Actual GCAS Trajectory')
plt.plot(t[:N], (mu), 'k', label='Expected GCAS Trajectory')
plt.plot(t[:N], (mu + 2*np.sqrt(va)), '--k', label='2$\sigma$ Interval')
plt.plot(t[:N], saturate(mu - 2*np.sqrt(va)), '--k')

plt.xlabel("Time [s]")
plt.ylabel("Altitude [ft]")
plt.title("GCAS Maneuver Trajectories")
plt.legend()
plt.grid()


In [ ]:
plt.figure(figsize=(8, 6))
nt = t[:N:100]
cov = akern.build_kernel_matrix(te.covariance, nt, nt)
vn = [te.mu(ti) for ti in t[:N:100]]
plt_vals = []
for i in range(0, 60):
    ys = np.random.multivariate_normal(vn, cov)
    #if (ys > 0.0).all():
    plt_vals.extend([nt, ys, "k"])
plt.grid()
plt.plot(*plt_vals, alpha=0.1)
plt.plot(nt, vn, '-.r', LineWidth=2.0)
plt.plot(nt, vn, '-.r', LineWidth=2.0, label='Posterior Mean')
#plot_data(ax[1], ptsn, velsn, dt=0.05)
plt.legend()
#plt.set_ylabel('x')
#plt.set_ylim(-1.5, 0.6)
plt.title('Posterior GCAS Realized Trajectories')
plt.xlabel("Time [s]")
plt.ylabel("Altitude [ft]")

In [ ]:
tes = [atraj.CubicSplineTrajectoryEstimator(11.0, 50.0, 50.0), 
       atraj.CubicSplineTrajectoryEstimator(11.0, 50.0, 50.0)]

In [ ]:
tt, xt, yt, zt = t[:nsl:st], x[:nsl:st], y[:nsl:st], z[:nsl:st]

# intent points
waypointsx = [[35,  1e+04, -400.0]]
waypointsy = [[35, 0.43e+04, 200]]

# uncertainty around intent
sigmas = [[50, 20]]

# training data
xd = np.diff(xt)/np.diff(tt)
yd = np.diff(yt)/np.diff(tt)

xd = np.hstack((xd[0], xd))
yd = np.hstack((yd[0], yd))

# append intent to training data
for (ti, xi, xdi), (ti, yi, ydi) in zip(waypointsx, waypointsy):
    tt = np.hstack((tt, ti))
    
    xt = np.hstack((xt, xi))
    xd = np.hstack((xd, xdi))
    
    yt = np.hstack((yt, yi))
    yd = np.hstack((yd, ydi))


In [ ]:
sigma = np.ones(len(tt))*0.01
sigmap = np.ones(len(tt))*0.01

for idx, (sigi, sigpi) in enumerate(sigmas[::-1]):
    sigma[-idx-1] = sigi
    sigmap[-idx-1] = sigpi

tes[0].fit(tt, np.vstack((xt, xd)).T, sigma=sigma, sigmap=sigmap)
tes[1].fit(tt, np.vstack((yt, yd)).T, sigma=sigma, sigmap=sigmap)

In [ ]:
mux = [tes[0].mu(ti) for ti in t[:N]]
vax = [tes[0].covariance(ti, ti) for ti in t[:N]]

In [ ]:
muy = [tes[1].mu(ti) for ti in t[:N]]
vay = [tes[1].covariance(ti, ti) for ti in t[:N]]

In [ ]:
plt.figure(figsize=(8, 6))
cm = plt.cm.get_cmap('rainbow')


muxr, muyr = mux[::1], muy[::1]
vaxr, vayr = vax[::1], vay[::1]
colors = np.linspace(0, 35, len(muxr))
sc = plt.scatter(muxr, muyr, s = np.array(np.sqrt(vaxr+vayr))*2, vmin=0, vmax=35, c=colors, cmap=cm, zorder=200)
#clb=plt.colorbar()
#sc.colorbar.set_label('Time [s]', rotation=270)

plt.axis("equal")
#plt.xlim(0, 12.e3)
#plt.ylim(-6e3, 6.e3)
#plt.ylim(-7e3, 7e3)
plt.grid()
plt.title("Waypoint Trajectory Distribution")
plt.xlabel("x [ft]")
plt.ylabel("y [ft]")
plt.colorbar(sc)
sc.colorbar.set_label('Time [s]', rotation=270)
plt.tight_layout()
plt.scatter([1e+04], [0.42972133e+04], c='k', marker='x', s=100, zorder=1000, label="Waypoint")
plt.scatter(xt[:-1], yt[:-1], c='k', s=20, zorder=1000, label="Current Path")
plt.legend()

In [ ]:
plt.plot(mux)
plt.plot(mux+np.sqrt(vax))
plt.plot(mux-np.sqrt(vax))